# OneHotEncode Target Values

"y" is 4 columns 

Tried 500 estimators
Then 200

In [1]:
#Import Dependencies

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
#SOURCE: https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database="Body_Performance", user='postgres', password='Challah6132', host='localhost', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)



Connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [3]:
body_df = pd.read_sql('select * from body_performance', con=conn)
body_df.head()

,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [4]:
#Closing the connection
conn.close()


In [5]:
#LabelEncoder on gender column
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

body_df['gender_num'] = labelencoder.fit_transform(body_df['gender'])

body_df.head(10)


,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking,gender_num
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C,1
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A,1
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C,1
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B,1
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B,1
5,36,F,165.4,55.40,22.0,64.0,119.0,23.8,21.0,27.0,153.0,B,0
6,42,F,164.5,63.70,32.2,72.0,135.0,22.7,0.8,18.0,146.0,D,0
7,33,M,174.9,77.20,36.9,84.0,137.0,45.9,12.3,42.0,234.0,B,1
8,54,M,166.8,67.50,27.6,85.0,165.0,40.4,18.6,34.0,148.0,C,1
9,28,M,185.0,84.60,14.4,81.0,156.0,57.9,12.1,55.0,213.0,B,1


In [6]:
#Using oneHotEncoder for class into 4 separate columns
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

encode2 = pd.DataFrame(enc.fit_transform(body_df.ranking.values.reshape(-1,1)))

encode2.columns = enc.get_feature_names_out(['ranking'])
encode2.head()

,ranking_A,ranking_B,ranking_C,ranking_D
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0


In [7]:
#Drop columns and merge encoded df

body_df = body_df.drop(["gender", "ranking"], axis=1)
body_df.head()


,age,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,gender_num
0,27,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,1
1,25,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,1
2,31,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,1
3,32,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1
4,28,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1


In [8]:
#Set X, y 
X = body_df.copy()
y = encode2

print(X.shape, y.shape)

(13393, 11) (13393, 4)


In [9]:
#Split train/test data and scale using Standard Scaler at this time (mean of 0). 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape)
print(X_test_scaled.shape)

(10044, 11)
(3349, 11)


### RandomForestClassifier

In [10]:
#Trying RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
y_pred = rf_clf.predict(X_test_scaled)

print(f'Training Score: {rf_clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {rf_clf.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.5915198566736339


In [11]:
feature_importances = rf_clf.feature_importances_
sorted(zip(rf_clf.feature_importances_, X.columns), reverse=True)

[(0.2556133126646863, 'sit_and_bend_forward_cm'),
 (0.1331023503869799, 'sit_ups_counts'),
 (0.09738121527626012, 'body_fat_pct'),
 (0.08914939741770866, 'age'),
 (0.08794752903164775, 'weight_kg'),
 (0.08159518833095651, 'gripforce'),
 (0.07555915163091356, 'broad_jump_cm'),
 (0.06516417793580569, 'height_cm'),
 (0.05203744673846165, 'systolic'),
 (0.04938708045845731, 'diastolic'),
 (0.01306315012812262, 'gender_num')]

In [12]:
#Accuracy score and confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions = rf_clf.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.5915198566736339


In [13]:
# printing confusion matrix but error      
cm = confusion_matrix(y_test.values.argmax(axis=1), predictions.argmax(axis=1))
#cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1", "Actual 2", "Actual 3"],
    columns=["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3"])
display(cm_df)

,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,803,15,0,0
Actual 1,494,307,35,9
Actual 2,323,63,421,28
Actual 3,131,12,49,659


In [14]:
#Classification report for random forest

print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.80      0.73      0.76       818
           1       0.77      0.36      0.49       845
           2       0.83      0.50      0.63       835
           3       0.95      0.77      0.85       851

   micro avg       0.85      0.59      0.70      3349
   macro avg       0.84      0.59      0.68      3349
weighted avg       0.84      0.59      0.68      3349
 samples avg       0.59      0.59      0.59      3349



C:\Users\mered\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Balanced Random Forest Classifier

In [16]:
#Trying RandomForestClassifier with OHE
from imblearn.ensemble import BalancedRandomForestClassifier

brf_clf = BalancedRandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)

print(f'Training Score: {brf_clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {brf_clf.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.5876381009256495


In [18]:
#Accuracy score and confusion Matrix

predictions2 = brf_clf.predict(X_test_scaled)
acc_score2 = accuracy_score(y_test, predictions2)
print(f"Accuracy Score : {acc_score2}")

Accuracy Score : 0.5876381009256495


In [19]:
# printing confusion matrix    
cm2 = confusion_matrix(y_test.values.argmax(axis=1), predictions2.argmax(axis=1))
#cm = confusion_matrix(y_test, predictions)
cm_df2 = pd.DataFrame(
    cm2, index=["Actual 0", "Actual 1", "Actual 2", "Actual 3"],
    columns=["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3"])
display(cm_df2)

,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,801,17,0,0
Actual 1,492,309,34,10
Actual 2,333,58,415,29
Actual 3,122,14,54,661


### Extra Trees Classifier

In [22]:
from sklearn.ensemble import ExtraTreesClassifier

et_clf = ExtraTreesClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)

print(f'Training Score: {et_clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {et_clf.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.5058226336219767


In [23]:
predictions3 = et_clf.predict(X_test_scaled)
acc_score3 = accuracy_score(y_test, predictions3)
print(f"Accuracy Score : {acc_score3}")

Accuracy Score : 0.5058226336219767


In [24]:
# printing confusion matrix    
cm3 = confusion_matrix(y_test.values.argmax(axis=1), predictions3.argmax(axis=1))
#cm = confusion_matrix(y_test, predictions)
cm_df3 = pd.DataFrame(
    cm3, index=["Actual 0", "Actual 1", "Actual 2", "Actual 3"],
    columns=["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3"])
display(cm_df3)

,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,808,10,0,0
Actual 1,662,151,27,5
Actual 2,441,31,337,26
Actual 3,164,7,40,640


#### RandomForestClassifier - 200 estimators

In [27]:
#Trying RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_clf2 = RandomForestClassifier(random_state=1, n_estimators=200).fit(X_train_scaled, y_train)
predictions4 = rf_clf2.predict(X_test_scaled)

print(f'Training Score: {rf_clf2.score(X_train_scaled, y_train)}')
print(f'Testing Score: {rf_clf2.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.585249328157659


In [28]:
# printing confusion matrix    
cm4 = confusion_matrix(y_test.values.argmax(axis=1), predictions4.argmax(axis=1))
#cm = confusion_matrix(y_test, predictions)
cm_df4 = pd.DataFrame(
    cm4, index=["Actual 0", "Actual 1", "Actual 2", "Actual 3"],
    columns=["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3"])
display(cm_df4)

,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,799,19,0,0
Actual 1,498,302,36,9
Actual 2,329,63,411,32
Actual 3,127,14,52,658


#### RandomForestClassifier - 1000 estimators

In [29]:
#Trying RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_clf3 = RandomForestClassifier(random_state=1, n_estimators=1000).fit(X_train_scaled, y_train)
predictions5 = rf_clf3.predict(X_test_scaled)

print(f'Training Score: {rf_clf3.score(X_train_scaled, y_train)}')
print(f'Testing Score: {rf_clf3.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.5882352941176471


In [30]:
# printing confusion matrix    
cm5 = confusion_matrix(y_test.values.argmax(axis=1), predictions5.argmax(axis=1))
#cm = confusion_matrix(y_test, predictions)
cm_df5 = pd.DataFrame(
    cm5, index=["Actual 0", "Actual 1", "Actual 2", "Actual 3"],
    columns=["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3"])
display(cm_df5)

,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,803,15,0,0
Actual 1,495,302,39,9
Actual 2,330,59,418,28
Actual 3,131,13,51,656
